# Databases SQL 2
# Gabriel Becton

In order to look at multi-table join queries, we are going to again create a simple, simple SQLite database.

In [ ]:
!pip install mechanize

In [ ]:
import sqlalchemy

The database will be stored in the file `boat_rental.db` in the local directory.

In [ ]:
# Delete the database if it already exists.
from pathlib import Path
dbfile = Path("boat_rental.db")
if dbfile.exists():
    dbfile.unlink()

sqlite_uri = "sqlite:///boat_rental.db"
sqlite_engine = sqlalchemy.create_engine(sqlite_uri)

Our boat rental buisness needs to store data in 3 tables, one holds information about the renters (sailors), a seconds holds information about the boats and the third gives details of rental transactions.

In [ ]:
sql_expr = """
CREATE TABLE sailors(
    sid INTEGER PRIMARY KEY, -- Sailor ID Number
    sname TEXT, -- Sailor Name
    rating INTEGER, -- Sailor skill rating
    age INTEGER); 

"""
result = sqlite_engine.execute(sql_expr)

In [ ]:
sql_expr = """
CREATE TABLE boats(
    bid INTEGER PRIMARY KEY, -- Boat ID Number
    bname TEXT, -- Boat Name
    bcolor TEXT);
"""
result = sqlite_engine.execute(sql_expr)

In [ ]:
sql_expr = """
CREATE TABLE reserves(
    sid INTEGER,
    bid INTEGER,
    day DATE,
    PRIMARY KEY(sid, bid, day),
    FOREIGN KEY (sid) REFERENCES sailors,
    FOREIGN KEY (bid) REFERENCES boats);
"""
result = sqlite_engine.execute(sql_expr)

Let's double-check the schema of our tables to make sure they are correct.

In [ ]:
inspector = sqlalchemy.inspect(sqlite_engine)
inspector.get_table_names()

In [ ]:
# Get column information
print('sailors = ')
for col in inspector.get_columns("sailors"):
    print(col)

print('boats = ')
for col in inspector.get_columns("boats"):
    print(col)
    
print('reserves = ')
for col in inspector.get_columns("reserves"):
    print(col)

Now let's populate our tables with data.

In [ ]:
sql_expr = """
INSERT INTO sailors VALUES 
  (1, 'Susan', 7, 24),
  (2, 'Jim', 2, 35),
  (3, 'Nancy', 8, 41),
  (4, 'Dustin', 7, 35),
  (5, 'Lubber', 8, 50),
  (6, 'Rusty', 10, 69);
  
"""
result = sqlite_engine.execute(sql_expr)

In [ ]:
sql_expr = """
INSERT INTO boats VALUES
  (101, 'Nina', 'red'),
  (102, 'Pinta', 'blue'),
  (103, 'Santa Maria', 'red');
"""
result = sqlite_engine.execute(sql_expr)

In [ ]:
sql_expr = """
INSERT INTO reserves VALUES
  (5, 102, '7/1/18'),
  (1, 103, '7/1/18'),
  (6, 101, '7/4/18'),
  (3, 102, '7/5/18');
"""
result = sqlite_engine.execute(sql_expr)

And display the information in our tables.

In [ ]:
import pandas as pd

sql_expr = """
SELECT * FROM sailors;
"""
pd.read_sql(sql_expr, sqlite_engine)

,sid,sname,rating,age
0,1,Susan,7,24
1,2,Jim,2,35
2,3,Nancy,8,41
3,4,Dustin,7,35
4,5,Lubber,8,50
5,6,Rusty,10,69


In [ ]:
sql_expr = """
SELECT * FROM boats;
"""
pd.read_sql(sql_expr, sqlite_engine)

,bid,bname,bcolor
0,101,Nina,red
1,102,Pinta,blue
2,103,Santa Maria,red


In [ ]:
sql_expr = """
SELECT * FROM reserves;
"""
pd.read_sql(sql_expr, sqlite_engine)

,sid,bid,day
0,5,102,7/1/18
1,1,103,7/1/18
2,6,101,7/4/18
3,3,102,7/5/18


### Make a simple, single table query
It may be helpful to look back at the previous Lab Notebook.

In [ ]:
# Perform a query of the sailors table to 
# display the names of the sailors who are of age 50
# or older and are, therefore, eligible for your company's 
# senior discount.

sql_expr = """
SELECT sname 
FROM sailors
WHERE age >= 50;

"""
pd.read_sql(sql_expr, sqlite_engine)

## Joins
If we want to look at information across tables, it is necessary to join them.

First let's look at the the result of performing the join.

In [ ]:
sql_expr = """
SELECT * -- all columns
FROM reserves, sailors;
"""
pd.read_sql(sql_expr, sqlite_engine)

,sid,bid,day,sid,sname,rating,age
0,5,102,7/1/18,1,Susan,7,24
1,5,102,7/1/18,2,Jim,2,35
2,5,102,7/1/18,3,Nancy,8,41
3,5,102,7/1/18,4,Dustin,7,35
4,5,102,7/1/18,5,Lubber,8,50
5,5,102,7/1/18,6,Rusty,10,69
6,1,103,7/1/18,1,Susan,7,24
7,1,103,7/1/18,2,Jim,2,35
8,1,103,7/1/18,3,Nancy,8,41
9,1,103,7/1/18,4,Dustin,7,35


The rows of the join are made up of all possible combinations of the rows of reserves and sailors.  We can get the same results by explicitly using the `JOIN` argument.

In [ ]:
sql_expr = """
SELECT * -- all columns
FROM reserves JOIN sailors;
"""
pd.read_sql(sql_expr, sqlite_engine)

,sid,bid,day,sid,sname,rating,age
0,5,102,7/1/18,1,Susan,7,24
1,5,102,7/1/18,2,Jim,2,35
2,5,102,7/1/18,3,Nancy,8,41
3,5,102,7/1/18,4,Dustin,7,35
4,5,102,7/1/18,5,Lubber,8,50
5,5,102,7/1/18,6,Rusty,10,69
6,1,103,7/1/18,1,Susan,7,24
7,1,103,7/1/18,2,Jim,2,35
8,1,103,7/1/18,3,Nancy,8,41
9,1,103,7/1/18,4,Dustin,7,35


### Natural/Inner Join
We will often want to join tables on an attribute that is shared accross tables.  For example, it makes sense to perform a join on sid for the reserves and sailors tables so that we are seeing reservation information for each sailor.  A natural or inner join only displays join rows where the join key (sid) is shared.  The following queries are all equivalent.

In [ ]:
sql_expr = """
SELECT *
FROM reserves AS r, sailors AS s
WHERE r.sid = s.sid
"""
pd.read_sql(sql_expr, sqlite_engine)

,sid,bid,day,sid,sname,rating,age
0,5,102,7/1/18,5,Lubber,8,50
1,1,103,7/1/18,1,Susan,7,24
2,6,101,7/4/18,6,Rusty,10,69
3,3,102,7/5/18,3,Nancy,8,41


In [ ]:
sql_expr = """
SELECT *
FROM reserves AS r JOIN sailors AS s
ON r.sid = s.sid
"""
pd.read_sql(sql_expr, sqlite_engine)

,sid,bid,day,sid,sname,rating,age
0,5,102,7/1/18,5,Lubber,8,50
1,1,103,7/1/18,1,Susan,7,24
2,6,101,7/4/18,6,Rusty,10,69
3,3,102,7/5/18,3,Nancy,8,41


In [ ]:
sql_expr = """
SELECT *
FROM reserves r INNER JOIN sailors s -- The AS arguments are always optional
ON r.sid = s.sid
"""
pd.read_sql(sql_expr, sqlite_engine)

,sid,bid,day,sid,sname,rating,age
0,5,102,7/1/18,5,Lubber,8,50
1,1,103,7/1/18,1,Susan,7,24
2,6,101,7/4/18,6,Rusty,10,69
3,3,102,7/5/18,3,Nancy,8,41


In [ ]:
# NATURAL JOIN, unlike the others, will automatically
# find column names both tables have in common
sql_expr = """
SELECT *
FROM reserves r NATURAL JOIN sailors s 
"""
pd.read_sql(sql_expr, sqlite_engine)

,sid,bid,day,sname,rating,age
0,5,102,7/1/18,Lubber,8,50
1,1,103,7/1/18,Susan,7,24
2,6,101,7/4/18,Rusty,10,69
3,3,102,7/5/18,Nancy,8,41


We can use natural joins to pull out information on the history of specific sailors with our company.

In [ ]:
sql_expr = """
SELECT s.sid, s.sname, r.bid, r.day
FROM reserves r, sailors s
WHERE r.sid = s.sid
AND s.age >= 30
ORDER BY s.sname
"""
pd.read_sql(sql_expr, sqlite_engine)

,sid,sname,bid,day
0,5,Lubber,102,7/1/18
1,3,Nancy,102,7/5/18
2,6,Rusty,101,7/4/18


A report on Rusty's buisness.

In [ ]:
sql_expr = """
SELECT s.sid, s.rating, r.bid, r.day
FROM reserves r JOIN sailors s
ON r.sid = s.sid
WHERE s.sname = 'Rusty'
"""
pd.read_sql(sql_expr, sqlite_engine)

,sid,rating,bid,day
0,6,10,101,7/4/18


We can also form inner joins on all 3 tables since both sailors and boats each share a column with reserves.

In [ ]:
sql_expr = """
SELECT *
FROM reserves r JOIN sailors s
ON r.sid = s.sid
JOIN boats b
ON r.bid = b.bid
"""
pd.read_sql(sql_expr, sqlite_engine)

,sid,bid,day,sid,sname,rating,age,bid,bname,bcolor
0,5,102,7/1/18,5,Lubber,8,50,102,Pinta,blue
1,1,103,7/1/18,1,Susan,7,24,103,Santa Maria,red
2,6,101,7/4/18,6,Rusty,10,69,101,Nina,red
3,3,102,7/5/18,3,Nancy,8,41,102,Pinta,blue


In [ ]:
# Generate a report on the company's boats.
# You should display the boat id number, boat name, boat color,
# and the sailor id, name, and day the sailor rented that boat.

sql_expr = """
SELECT s.sid, s.sname, r.day, b.bid, b.bname, b.bcolor
FROM reserves r JOIN sailors s
ON r.sid = s.sid
JOIN boats b
ON r.bid = b.bid
"""
pd.read_sql(sql_expr, sqlite_engine)


### Left/Right Joins

Left joins will include all entries by key in the left table and fill in null values for the right table, where necessary.

In [ ]:
# This returns the same information as an inner join since
# all sid values in the reserves table have a corresponding
# entry in the sailors table

sql_expr = """
SELECT *
FROM reserves r LEFT JOIN sailors s
ON r.sid = s.sid
"""
pd.read_sql(sql_expr, sqlite_engine)

In [ ]:
# But not all sailors have rented a boat

sql_expr = """
SELECT *
FROM sailors s LEFT JOIN reserves r
ON r.sid = s.sid
"""
pd.read_sql(sql_expr, sqlite_engine)

Right joins work the same way (this time taking all rows from the right table), but SQLite does not support them.  `FULL JOIN` performs an outer join where each row of each table is represented and nulls are filled in, where appropriate.  SQLite does not support outer joins but many other SQL versions do.

## SQL Queries and NULL values

As we see in the example above, left/right/outer joins can result in Null values being inserted into the resulting query result table.  

It is also possible that Null values can result from missing information.

In [ ]:
sql_expr = """
INSERT INTO sailors VALUES
  (7, 'Jack Sparrow', NULL, 35);  -- no skill rating available
"""
result = sqlite_engine.execute(sql_expr)

sql_expr = """
SELECT * 
FROM sailors;
"""
pd.read_sql(sql_expr, sqlite_engine)

The following will result in no output because any operator acting on a null value returns a null output.

In [ ]:
sql_expr = """
SELECT *
FROM sailors
WHERE rating = NULL
"""
pd.read_sql(sql_expr, sqlite_engine)

Jack will also never show in a query that references the rating value.

In [ ]:
sql_expr = """
SELECT *
FROM sailors
WHERE rating < 8
"""
pd.read_sql(sql_expr, sqlite_engine)

You must do an explicit NULL check to find null values.

In [ ]:
sql_expr = """
SELECT *
FROM sailors
WHERE rating IS NULL
"""
pd.read_sql(sql_expr, sqlite_engine)

In [ ]:
sql_expr = """
SELECT *
FROM sailors
WHERE rating IS NOT NULL
"""
pd.read_sql(sql_expr, sqlite_engine)

In [ ]:
sql_expr = """
SELECT *
FROM sailors

"""
pd.read_sql(sql_expr, sqlite_engine)

In [ ]:
sql_expr = """
SELECT *
FROM boats

"""
pd.read_sql(sql_expr, sqlite_engine)

In [ ]:
sql_expr = """
SELECT *
FROM reserves

"""
pd.read_sql(sql_expr, sqlite_engine)

In [ ]:
# Generate a report that shows the ID, Name, and Age of all sailors who 
#         never rented a boat

sql_expr = """
SELECT s.sid, s.sname, s.age
FROM sailors s LEFT JOIN reserves r
ON r.sid = s.sid
WHERE bid IS NULL
"""
pd.read_sql(sql_expr, sqlite_engine)


,sid,sname,age
0,2,Jim,35
1,4,Dustin,35
2,7,Jack Sparrow,35
